In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from cleanco import basename

In [2]:
headers = {'User-Agent': "mhye@ucsd.edu"}

In [3]:
def get_cik(ticker, headers=headers):
    ticker = ticker.upper().replace(".", "-")
    ticker_json = requests.get(
        "https://www.sec.gov/files/company_tickers.json", headers=headers
    ).json()

    for company in ticker_json.values():
        if company["ticker"] == ticker:
            cik = str(company["cik_str"]).zfill(10)
            return cik
    raise ValueError(f"Ticker {ticker} not found in SEC database")

def filing_finder(cik, headers=headers):
    headers = headers
    url = f"https://data.sec.gov/submissions/CIK{cik}.json"
    company_json = requests.get(url, headers=headers).json()
    company_df = pd.DataFrame(company_json["filings"]["recent"])
    df = company_df[company_df["form"] == "4"]
    return df

def roster_finder(ticker):
    cik = get_cik(ticker)
    filings = filing_finder(cik, headers)
    insider_names = set()
    
    # Loop through each filing in the DataFrame
    for _, row in filings.iterrows():
        # if len(insider_names) >= 15:
        #     break
        accession_number = row['accessionNumber'].replace("-", "")
        url = f"https://www.sec.gov/Archives/edgar/data/{cik}/{accession_number}/{row['primaryDocument']}"
        
        # Request the HTML form
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            name = soup.find('a')
            insider_names.add(name.text.lower())
    
    return(insider_names)

In [4]:
roster_finder('T')

{'arroyo f. thaddeus',
 'at&t inc.',
 'desroches pascal',
 'di piazza samuel a jr.',
 'dial debra l.',
 'fisher richard w',
 'ford scott t',
 'gillespie edward w',
 'huntley david s.',
 'hutchins glenn h',
 'kennard william e',
 'kilar jason',
 'lee debra l',
 'lee lori m',
 'legg jeremy alan',
 'luczo stephen j',
 'mayer marissa a',
 'mcatee david r ii',
 'mccallister michael b',
 'mcelfresh jeffery s.',
 'mooney beth e',
 'rose matthew k',
 'sabrina sanders s',
 'santone angela',
 'smith kenny kellyn',
 'stankey john t',
 'stephens john joseph',
 'stephenson randall l',
 'taylor cindy b',
 'ubinas luis a',
 'yang geoffrey y'}

In [5]:
roster_finder('TSLA')

{'ahuja deepak',
 'baglino andrew d',
 'branderiz eric',
 'buss brad w',
 'denholm robyn m',
 'ehrenpreis ira matthew',
 'ellison lawrence joseph',
 'field john douglas',
 'gracias antonio j.',
 'guillen jerome m',
 'jurvetson stephen t',
 'kirkhorn zachary',
 'mcneill jon',
 'mizuno hiromichi',
 'murdoch james r',
 'musk elon',
 'musk kimbal',
 'rice linda johnson',
 'straubel jeffrey b',
 'taneja vaibhav',
 'wilson-thompson kathleen',
 'zhu xiaotong'}